<a href="https://colab.research.google.com/github/uncoded-ro/lp2/blob/main/modul_6/lp2_m6_01_programare_in_retea.ipynb"
 target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"
 style="float: right;"/></a>

# Limbaje de programare 2
**Modul 6 - Programare in retea in Python**

*autor notebook:* [Bogdan Dragulescu](https://ti.etcti.upt.ro/bogdan-dragulescu/)<br/>
*continut original:* [Bogdan Dragulescu](https://ti.etcti.upt.ro/bogdan-dragulescu/),
*sub format text* in cadrul cursului pe [cv.upt.ro](http://cv.upt.ro)

## Obiective

* Utilizarea API-ului de nivel jos pentru a transmite date
* Utilizarea urllib pentru citirea fișierelor din WWW
* Biblioteci externe pentru procesarea documentelor HTML


## Introducere

Socket-urile permit comunicarea intre doua procese diferite pe aceeasi masina sau pe masini diferite. Este o
modalitate de a comunica intre calculatoare utilizand standardul Unix file descriptors. In sistemele de operare de tip
 Unix orice actiune de tip I/O (intrare/iesire) este realizata prin scrierea sau citirea intr-un file descriptor.
 Acesta este doar un intreg asociat cu un fisier deschis si poate fi o conexiune, un fisier text, un terminal, etc.

Au fost utilizate initial in 1971 in ARPANET, iar apoi au fost introduse in API in BSD (Berkeley Software Distribution)
in 1983. Cand internetul a prins avant in 1990 prin WWW, a crescut si cererea pentru programarea in retea. Chiar daca
protocoalele utilizate in retea au evoluat, API-ul de nivel jos a ramas la fel.

Un socket Unix este utilizat in realizarea aplicatiilor client-server. Un server este un proces care executa anumite
operatii pe baza unei cereri de la client. Marea majoritate a protocoalelor de la nivelul aplicatiilor cum ar fi FTP,
SMTP, POP3 folosesc sockets pentru a realiza conexiunea intre client si server, iar apoi utilizeaza acea conexiune
pentru a schimba date.
Utilizatorii au disponibile 4 tipuri de socket-uri. Dintre acestea doua sunt cele mai utilizate:
* **Stream sockets** - transmiterea in retea este garantata. Daca se transmite in retea “A, B, C” acestea vor ajunge la
client in aceeasi ordine. Aceste tipuri de sockets utilizeaza TCP (Transmission Control Protocol) pentru a transfera
datele. Daca nu se pot transmite datele se va returna o eroare la emitator.
* **Datagram Sockets** - transmiterea in retea nu este garantata. Nu este necesara o conexiune deschisa ca si in cazul
Stream Sockets - se construieste pachetul cu informatia despre destinatie si este transmis in retea. Aceste socket-uri
folosesc UDP(User Datagram Protocol).

Marea majoritatea a aplicatiilor de retea utilizeaza o arhitectura de tip client-server. Aceasta arhitectura este
compusa din doua procese (sau doua aplicatii) care comunica intre ele pentru a schimba informatie. Un proces va avea
functia de client, iar celalalt functia de server.

## SOCKETS

Modulul din Python care permite constructia de aplicatii cu socket-uri poarta acelasi nume: socket. Acesta creaza o
interfata pentru Berkeley sockets API. Documentatia pentru biblioteca din limbaj o gasiti la adresa:
[link](https://docs.python.org/3/library/socket.html).

Principalele functii si metode din acest modul sunt:
* socket() - creaza un obiect de tip socket
* bind() - asociaza socket-ul cu o anumita interfata
* listen() - asculta conexiunile catre socket
* accept() - accepta o conexiune catre un socket
* connect() - se conecteaza la un socket de la adresa specificata
* connect_ex() - functionalitate similara cu metoda connect(), diferenta: returneaza un cod de eroare in loc de o
exceptie
* send() - trimite date prin socket
* recv() - primeste date de la socket
* close() - inchide socket-ul

Dupa cum am amintit sunt doua tipuri de socket-uri utilizate in general:
* **Stream socket** - acestea pot fi create specificand tipul socket-ului prin socket.SOCK_STREAM. (Utilizeaza TCP)
* **Datagram socket** - acestea pot fi create specificand tipul socket-ului prin socket.SOCK_DGRAM. (Utilizeaza UDP)

Obs!  Indicat sa utilizam TCP pentu a scapa de grija transmiterii complete si corecte a pachetelor. Echipamantele de
retea, de exemplu routerele au limitari (CPU, RAM, etc) la fel ca severele si clientii.

Diagrama si explicatiile ce descriu secventa apelurilor metodelor intr-o topologie client-server o gasiti in documentul
original in cadrul cursului pe [cv.upt.ro](http://cv.upt.ro).

Un exemplu de aplicatie de tip server este reprezentata mai jos (server.py).
Metoda socket.socket() creaza un obiect de tip socket care suporta “context manager type” si se poate utiliza cu
instructiunea with. In acest mod nu este necesar apelul metodei close() la finalul aplicatiei. Primeste ca si
argumente clasa de adrese prin intermediul constantei socket.AF_INET ce reprezinta IPV4, respectiv tipul socket-ului
pentru TCP prin constanta socket.SOCK_STREAM.

```python
###############
## server.py ##
###############
import socket

HOST = '127.0.0.1'
PORT = 12345

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print('Server | Conectat de la adresa', addr)
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)
```

In interiorul blocului with sunt utilizate urmatoarele metode:
* **bind()** - este utilizata pentru a asocia socket-ul cu o anumita interfata si un anumit port. Pentru IPV4 aceasta
este specificata intr-un tuplu ce contine adresa ip si portul.
* **listen()** -  activeaza serverul pentru a primi conexiuni. Transforma socket-ul intr-un socket care asculta.
* **accept()**  - blocheaza si asteapta o conexiune de intrare. Cand un client se conecteaza returneaza un nou obiect
de tip socket care reprezinta conexiunea si un tuplu in care este pastrata adresa clientului. Acest nou socket este
utilizat pentru a comunica cu clientul si este diferit fata de socket-ul de ascultare.

Dupa ce am obtinut socket-ul de conectare o bucla infinita este utilizata pentru a parcurge valorile returnate de
metoda recv(). Aceasta citeste ce trimite clientul si se retransmit datele inapoi la client utilizand
metoda conn.sendall().

Daca metoda recv() returneaza un obiect gol b’’ (un obiect binar gol), atunci clientul opreste conexiunea.

Un exemplu de aplicatie de tip client ce comunica cu serverul este reprezentata mai jos (client.py).

```python
###############
## client.py ##
###############
import socket

HOST = '127.0.0.1'
PORT = 12345

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall('Salutare natiune!'.encode())
    data = s.recv(1024)

print('Client | Am primit raspuns', data.decode())
```

Se foloseste instructiunea with pentru a gestiona inchiderea socket-ului dupa utilizare. Se utilizeaza metoda
connect() pentru a realiza o conexiunea la server in care este specificata adresa si portul server-ului. Clientul
foloseste metoda sendall() pentru a trimite tot mesajul catre server. La final mesajul receptional este citit
prin metoda recv() si este afisat.

In [ ]:
# Descarcarea fisierelor server.py si client.py
!wget https://raw.githubusercontent.com/uncoded-ro/lp2/main/modul_6/server.py
!wget https://raw.githubusercontent.com/uncoded-ro/lp2/main/modul_6/client.py

Pentru a testa comunicarea intre server si client vom apela in aceeasi instructiune executia celor doua fisiere.

*ATENTIE! Este posibil ca apelul sa nu functioneze corect de fiecare data. La prima utilizare ar trebui sa ruleze
corect*.<br>
In cazul in care nu functioneaza rularea puteti testa comunicarea client-server creand pe calculatorul propriu
fisierele server.py si client.py. Deschideti doua ferestre de terminal, apelati in prima fereastra server.py iar in a
doua client.py.

In [ ]:
!python3 server.py & python3 client.py &

## Browser web. URLLIB.

Aceeasi arhitectura client-server este utilizata si in aplicatiile web. In acest caz in rolul clientului sunt
aplicatiile de tip browser web. Putem utiliza socket-uri pentru a scrie cel mai simplu browser web. Pentru a face
acest lucru trebuie sa respectam protocolul HTTP pentru a trimite o cerere catre server, iar acesta va trimite un
raspuns.

In prima faza clientul realizeaza conexiunea catre server folosind aceeasi sintaxa utilizand socket(). Pentru ca vrem
sa realizam un browser web, protocolul HTTP ne spune ca trebuie sa trimitem o comanda de tip GET urmata de o linie
goala. Dupa ce am trimis linia goala se primeste un raspuns pe care il vom afisa. Cand metoda recv() nu returneaza
nimic (am procesat tot documentul primit ca raspuns), programul opreste executia.

In [ ]:
###################
## client_web.py ##
###################
import socket

HOST = 'upt.ro'
PORT = 80

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
   s.connect((HOST, PORT))
   cmd = 'GET https://upt.ro HTTP/1.0\r\n\r\n'.encode()
   s.sendall(cmd)
   while True:
       data = s.recv(1024)
       if not data:
           break
       print(data.decode())

Ca parte din libraria standard, Python ofera si clase care fac folosirea mai usoara a acestor functii de nivel jos.
Sunt disponibile in Python multe module care se bazeaza pe aceasta clasa socket de nivel jos care permite implementarea
unor protocoale de nivel superior cum ar fi HTTP, FTP sau SMTP. O lista completa de module ce implementeaza protocoale
internet o gasiti la adresa: [link](https://docs.python.org/3/library/internet.html)

Unul dintre acestea este urllib. Folosind acest modul putem trata o pagina web ca un simplu fisier. Trebuie sa indicam
ce pagina web vrem sa folosim iar biblioteca urllib va utiliza protocolul HTTP pentru a obtine documentul cerut.
Exemplul precedent este rescris utilizand acest modul.

In [ ]:
##########################
## client_web_urllib.py ##
##########################
import urllib.request

with urllib.request.urlopen('https://upt.ro') as p:
    for line in p:
        print(line.decode().strip())

Obs!  Obiectul returnat de urlopen() are disponibila metoda read() pentru a citi complet pagina web. Se poate utiliza
pentru a citi resurse binare cum ar fi o fotografie.

## Procesarea documentelor HTML

Urllib este utilizat des pentru a realiza un ‘scraper web’. Un scraper este un program care pretinde a fi un browser
web si returneaza paginile html, apoi examineaza datele returnate pentru a identifica sabloane.

Un exemplu de astfel de aplicatii este un motor de cautare, de exemplu Google. Acesta extrage linkurile din pagini
pentru a descoperii alte pagini si a construi un algoritm de ordonare.
Avand exemplul precedent de citire a unei pagini web am putea dezvolta aplicatia astfel incat sa extragem toate
linkurile din pagina returnata si numaram aparitia lor.

**Cum am face acest lucru?**

O varianta ar fi sa utilizam expresii regulate. Din nefericire paginile HTML nu respecta tot timpul strictetea XML si
este foarte probabil ca utilizand doar expresii regulate anumite linkuri sa nu fie gasite sau sa fie returnate partial.

O solutie la aceasta problema este utilizarea unei biblioteci specializate pentru procesarea documentelor HTML. Un
exemplu de astfel de biblioteca este BeatifulSoup4.

Exemplul de mai jos va afisa toate adresele href identificate in tagul \<a\>. BeatifulSoup() primeste documentul ce
trebuie sa fie procesat si se specifica tipul parser-ului utilizat (in cazul acesta html.parser). Obiectul soup poate
fi folosit pentru a specifica ce tag de marcaj vrem sa extragem din ‘supa’ si returneaza o lista cu elementele
identificate.
Pentru fiecare element identificat putem utiliza metode/atribute de extragere a informatiei dorite: get(‘href’)
(returneaza atribului specificat), attrs (returneaza atributele din tag-ul <a ….>), contents (returneaza tot tag-ul de
la \<a\> la \</a\>).

In [ ]:
####################
## client_soup.py ##
####################
import urllib.request
from bs4 import BeautifulSoup

html = urllib.request.urlopen('https://upt.ro').read()
soup = BeautifulSoup(html, 'html.parser')

tags = soup('a')
for tag in tags:
    print(tag.get('href'))

Documentatia completa a modului BeautifulSoup o gasiti la adresa 
[link](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)